In [44]:
import pickle
import numpy as np
import pandas as pd
import networkx as nx
import re
from sklearn.preprocessing import scale
%matplotlib inline

# load

In [2]:
with open("nodeMeta.pkl", 'rb') as f:
    nodeMeta = pickle.load(f)

In [41]:
prefix = "4step_"
df = pd.read_csv(prefix+"sample.csv")
X = np.squeeze(np.dstack([df['head'].values, df['tail'].values]))
Y = df['label'].values
print(X.shape, Y.shape)

(169941, 2) (169941,)


In [4]:
test_x = []
for fromId, toId in np.loadtxt('t3-test.txt'):
    test_x.append((fromId,  toId))
test_x = np.array(test_x).astype('int64')

In [5]:
prefix = "4step_"
with open(prefix+'feat.pkl', 'rb') as f:
    feat = pickle.load(f)
with open(prefix+'test_f.pkl', 'rb') as f:
    test_f = pickle.load(f)

# gen

# extra features

In [6]:
def get_features(s):
    width = 3
    s = s.lower()
    return [s[i:i+width] for i in range(max(len(s)-width+1, 1))]

def add_time_sim_feat(h, t):
    cos1 = cosine(nodeMeta[h]["tvec"], nodeMeta[t]["tvec"])
    cos2 = cosine(nodeMeta[h]["avec"], nodeMeta[t]["avec"])
    delta = (nodeMeta[t]['date'] - nodeMeta[h]['date']).days
    time1 = [nodeMeta[t]['date'].day, nodeMeta[t]['date'].month, nodeMeta[t]['date'].year]
    time2 = [nodeMeta[h]['date'].day, nodeMeta[h]['date'].month, nodeMeta[h]['date'].year]
    
    s1, s2 = set(nodeMeta[h]['title']), set(nodeMeta[t]['title'])
    jaccard1 = float(len(s1.intersection(s2)))/float(len(s1.union(s2)))
    s1, s2 = set(nodeMeta[h]['abstract']), set(nodeMeta[t]['abstract'])
    jaccard2 = float(len(s1.intersection(s2)))/float(len(s1.union(s2)))
    
#     a = get_features(" ".join(nodeMeta[h]['title']))
#     b = get_features(" ".join(nodeMeta[t]['abstract']))
#     ham_dist1 = Simhash(a).distance(Simhash(b))
#     a = get_features(" ".join(nodeMeta[h]['title']))
#     b = get_features(" ".join(nodeMeta[t]['abstract']))
#     ham_dist2 = Simhash(a).distance(Simhash(b))
    
    return [cos1, cos2, delta, jaccard1, jaccard2] + time1 + time2# + [ham_dist1, ham_dist2]

In [7]:
%%time

G_train = nx.read_edgelist('t3-train.txt', create_using=nx.DiGraph(), nodetype=int, edgetype=int)
G_test = nx.read_edgelist('t3-test.txt', create_using=nx.DiGraph(), nodetype=int, edgetype=int)
unG_train = nx.read_edgelist('t3-train.txt', create_using=nx.Graph(), nodetype=int, edgetype=int)
unG_test = nx.read_edgelist('t3-test.txt', create_using=nx.Graph(), nodetype=int, edgetype=int)

# check
from_node = set()
to_node = set()
for i in G_test.edges:
    from_node.add(i[0])
    to_node.add(i[1])

print("from")
print(len(from_node), "from node")
print(len(from_node & set(G_train.nodes))/len(from_node), "in train")
print("to")
print(len(to_node), "to node")
print(len(to_node & set(G_train.nodes))/len(to_node), "in train")

# compute
deg_cent = nx.degree_centrality(G_train)
in_cent = nx.in_degree_centrality(G_train)
out_cent = nx.out_degree_centrality(G_train)
b_cent = nx.betweenness_centrality(G_train)
l_cent = nx.load_centrality(G_train)
tri = nx.triangles(unG_train)
kc = nx.katz_centrality(G_train)
page = nx.pagerank(G_train)


#store
with open("deg_cent.pkl", 'wb') as f:
    pickle.dump(deg_cent, f)
with open("in_cent.pkl", 'wb') as f:
    pickle.dump(in_cent, f)
with open("out_cent.pkl", 'wb') as f:
    pickle.dump(out_cent, f)
with open("b_cent.pkl", 'wb') as f:
    pickle.dump(b_cent, f)
with open("l_cent.pkl", 'wb') as f:
    pickle.dump(l_cent, f)
with open("tri.pkl", 'wb') as f:
    pickle.dump(tri, f)
with open("kc.pkl", 'wb') as f:
    pickle.dump(kc, f)
with open("page.pkl", 'wb') as f:
    pickle.dump(page, f)

from
3246 from node
0.0046210720887245845 in train
to
9213 to node
0.7761858243785955 in train
CPU times: user 6min 17s, sys: 120 ms, total: 6min 17s
Wall time: 6min 17s


In [29]:
with open("deg_cent.pkl", 'rb') as f:
    deg_cent = pickle.load(f)
with open("in_cent.pkl", 'rb') as f:
    in_cent = pickle.load(f)
with open("out_cent.pkl", 'rb') as f:
    out_cent = pickle.load(f)
with open("b_cent.pkl", 'rb') as f:
    b_cent = pickle.load(f)
with open("l_cent.pkl", 'rb') as f:
    l_cent = pickle.load(f)
with open("tri.pkl", 'rb') as f:
    tri = pickle.load(f)
with open("kc.pkl", 'rb') as f:
    kc = pickle.load(f)
with open("page.pkl", 'rb') as f:
    page = pickle.load(f)

# add only to_node
def add_node_feature(h, t):
    #feat = list(embedding[node])
    feat = []
    feat.append(G_train.degree(t) if t in G_train.nodes else 0)
    feat.append(G_train.in_degree(t) if t in G_train.nodes else 0)
    feat.append(G_train.out_degree(t) if t in G_train.nodes else 0)
    
    feat.append(deg_cent[t] if t in deg_cent else 0)
    feat.append(in_cent[t] if t in in_cent else 0)
    feat.append(out_cent[t] if t in out_cent else 0)
    feat.append(b_cent[t] if t in b_cent else 0)
    feat.append(l_cent[t] if t in l_cent else 0)
    feat.append(tri[t]  if t in tri else 0)
    feat.append(kc[t]  if t in kc else 0)
    feat.append(page[t]  if t in page else 0)
    return feat

In [8]:
def substitute(idx1, idx2, org, f):
    assert f.shape[1] == (idx2-idx1+1)
    assert f.shape[0] == org.shape[0]
    org[:, idx1:idx2+1] = f

# add

In [45]:
##################
extract_feat = add_node_feature
##################

extra_feat = []
for h, t in test_x:
    extra_feat.append(extract_feat(h, t))

for h, t in X:
    extra_feat.append(extract_feat(h, t))

extra_feat = scale(np.array(extra_feat))
print(extra_feat.shape)

(244739, 11)


In [46]:
test_f = np.hstack([test_f, extra_feat[:len(test_x)]])
feat = np.hstack([feat, extra_feat[len(test_x):]])
print(test_f.shape, feat.shape)

(74798, 1624) (169941, 1624)


In [47]:
prefix = "4step_"
with open(prefix+'feat.pkl', 'wb') as f:
    pickle.dump(feat, f)
with open(prefix+'test_f.pkl', 'wb') as f:
    pickle.dump(test_f, f)